In [1]:
from malay_stopwords import stopwords, stopwords_1, _list_laugh
from malay_normalization import rules_normalizer, sounds
from malay_lexicon_polarity import neg_Words,pos_Words,translated_pos_words,translated_neg_words
import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter('ignore')

import numpy as np
import json
import re
from unidecode import unidecode
import itertools

In [2]:
#analyzing method
pos ='1'
neg ='-1'
neu ='0'
pos_senti = set(list(pos_Words))
neg_senti = set(list(neg_Words))
def sentiment(words):
    pslen= len(pos_senti.intersection(words))
    nglen= len(neg_senti.intersection(words))
    
    if pslen > nglen:
        return pos
    elif pslen < nglen:
        return neg
    else:
        return neu

In [3]:
#analyzing method
from collections import Counter

def count_senti(sentences):
    sents = Counter()
    words = Counter()
    
    for sentence in sentences:
        senti = sentiment(sentence)
        sents[senti] += 1
        words[senti]+= len(sentence)
    return sents,words

In [4]:
#analyzing overall sentiment
import nltk
def parse_senti(text):
    
    sentences = [
        [word.lower() for word in nltk.word_tokenize(sentence)]
        for sentence in nltk.sent_tokenize(text)
    ]
    
    sents, words = count_senti(sentences)
    total = sum(words.values())
    
    for sentiment, count in words.items():
        pcent = (count / total) * 100
        nsents = sents[sentiment]
        
        return(
            sentiment
        )

In [5]:
#pre-processing normalization
def _dict_replace(wordlist, _dict):
        return [_dict.get(w, w) for w in wordlist]

def cleaning2(e_texts,rules):
    text1 = e_texts.split()   
    text1 = _dict_replace(text1, rules)
    text1 = ' '.join(text1)
    return text1

In [6]:
#preprocessing removing stopwords & symbol
STOPWORDS = set(stopwords + stopwords_1 + _list_laugh)
def clean_tweet(tweet):
    comments = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|[^a-zA-Z#]"," ",str(tweet)).split())
    comments = comments.lower().split()
    comments = ' '.join([word for word in comments if word not in STOPWORDS])
    return comments

In [7]:
#final improvement
def negate(self,text):

    negation = False
    delims = "?.,!:;"
    result = []
    words = text.split()
    

    for word in words:
        stripped = word.strip(delims).lower()
        negated = "tak_" + stripped if negation else stripped
        result.append(negated)
        negation = False

        if any(neg in word for neg in ["tak","bukan", "Tak","tidak","kurang"]):
            negation = not negation

    final= ' '.join(result)

    return final

In [8]:
import pandas as pd
df= pd.read_csv('restaurant review comments.csv',encoding = "ISO-8859-1")

In [9]:
df.head()

,tweet,senti
0,turn off betul kedai makan bancuh air tak seda...,-1
1,Mende tah aku makan tadi. Tak sedap. *sebab bo...,-1
2,"@nikasyrvf Mestii, sini kedai makan bukan seda...",-1
3,Tapi milo dekat fast food restaurant tak sedap...,-1
4,Semua makanan bau tengik. Loya tekak dibuatnya...,-1


In [10]:
import numpy as np
df['tweets']=np.array([clean_tweet(text) for text in df.tweet])
df.head(5)

,tweet,senti,tweets
0,turn off betul kedai makan bancuh air tak seda...,-1,turn off makan bancuh air tak sedap ni tak pay...
1,Mende tah aku makan tadi. Tak sedap. *sebab bo...,-1,mende tah makan tak sedap claim makan mengarut
2,"@nikasyrvf Mestii, sini kedai makan bukan seda...",-1,mestii makan bukan sedap haha
3,Tapi milo dekat fast food restaurant tak sedap...,-1,milo fast food restaurant tak sedap
4,Semua makanan bau tengik. Loya tekak dibuatnya...,-1,bau tengik loya tekak ape teruk peel demam pre...


In [11]:
import numpy as np
df['tweets']=np.array([cleaning2(text,sounds) for text in df.tweets])
df.head(5)

,tweet,senti,tweets
0,turn off betul kedai makan bancuh air tak seda...,-1,turn off makan bancuh air tak sedap ni tak pay...
1,Mende tah aku makan tadi. Tak sedap. *sebab bo...,-1,mende tah makan tak sedap claim makan mengarut
2,"@nikasyrvf Mestii, sini kedai makan bukan seda...",-1,mestii makan bukan sedap haha
3,Tapi milo dekat fast food restaurant tak sedap...,-1,milo fast food restaurant tak sedap
4,Semua makanan bau tengik. Loya tekak dibuatnya...,-1,bau tengik loya tekak ape teruk peel demam pre...


In [12]:
df['tweets']=np.array([cleaning2(text,rules_normalizer) for text in df.tweets])
df.head(5)

,tweet,senti,tweets
0,turn off betul kedai makan bancuh air tak seda...,-1,turn off makan bancuh air tak sedap ini tak pa...
1,Mende tah aku makan tadi. Tak sedap. *sebab bo...,-1,mende entah makan tak sedap claim makan mengarut
2,"@nikasyrvf Mestii, sini kedai makan bukan seda...",-1,mestii makan bukan sedap haha
3,Tapi milo dekat fast food restaurant tak sedap...,-1,milo fast food restaurant tak sedap
4,Semua makanan bau tengik. Loya tekak dibuatnya...,-1,bau tengik loya tekak apa teruk peel demam pre...


In [13]:
df['without_valenceshifter'] = df['tweets'].apply(parse_senti)
df.head(20)

,tweet,senti,tweets,without_valenceshifter
0,turn off betul kedai makan bancuh air tak seda...,-1,turn off makan bancuh air tak sedap ini tak pa...,1
1,Mende tah aku makan tadi. Tak sedap. *sebab bo...,-1,mende entah makan tak sedap claim makan mengarut,0
2,"@nikasyrvf Mestii, sini kedai makan bukan seda...",-1,mestii makan bukan sedap haha,1
3,Tapi milo dekat fast food restaurant tak sedap...,-1,milo fast food restaurant tak sedap,1
4,Semua makanan bau tengik. Loya tekak dibuatnya...,-1,bau tengik loya tekak apa teruk peel demam pre...,-1
5,Aku rasa aku kena mula jaga pemakanan. Gastrik...,-1,kena jaga pemakanan gastrik ini teruk makan ke...,-1
6,Apa dia letak sia kat makanan aku. Balik sakit...,-1,letak sia sakit kepala teruk ini,-1
7,@KuaiMuhamad Tak jgk.. Lama gak x kena food po...,-1,tak juga gak tidak kena food poisoning just my...,-1
8,LoL jual makanan tak sedap pastu mahal lak tu ...,-1,jual tak sedap mahal pula itu haiss,0
9,Makanan dia tak berapa sedap la for me. Dekat ...,-1,tak sedap lah untuk me surau disediakan,1


In [14]:
df['negation']=np.array([negate(self=None,text=text) for text in df.tweets])
df.head(5)

,tweet,senti,tweets,without_valenceshifter,negation
0,turn off betul kedai makan bancuh air tak seda...,-1,turn off makan bancuh air tak sedap ini tak pa...,1,turn off makan bancuh air tak tak_sedap ini ta...
1,Mende tah aku makan tadi. Tak sedap. *sebab bo...,-1,mende entah makan tak sedap claim makan mengarut,0,mende entah makan tak tak_sedap claim makan me...
2,"@nikasyrvf Mestii, sini kedai makan bukan seda...",-1,mestii makan bukan sedap haha,1,mestii makan bukan tak_sedap haha
3,Tapi milo dekat fast food restaurant tak sedap...,-1,milo fast food restaurant tak sedap,1,milo fast food restaurant tak tak_sedap
4,Semua makanan bau tengik. Loya tekak dibuatnya...,-1,bau tengik loya tekak apa teruk peel demam pre...,-1,bau tengik loya tekak apa teruk peel demam pre...


In [15]:
df['with_valenceshifter'] = df['negation'].apply(parse_senti)
df.head(20)

,tweet,senti,tweets,without_valenceshifter,negation,with_valenceshifter
0,turn off betul kedai makan bancuh air tak seda...,-1,turn off makan bancuh air tak sedap ini tak pa...,1,turn off makan bancuh air tak tak_sedap ini ta...,-1
1,Mende tah aku makan tadi. Tak sedap. *sebab bo...,-1,mende entah makan tak sedap claim makan mengarut,0,mende entah makan tak tak_sedap claim makan me...,-1
2,"@nikasyrvf Mestii, sini kedai makan bukan seda...",-1,mestii makan bukan sedap haha,1,mestii makan bukan tak_sedap haha,-1
3,Tapi milo dekat fast food restaurant tak sedap...,-1,milo fast food restaurant tak sedap,1,milo fast food restaurant tak tak_sedap,-1
4,Semua makanan bau tengik. Loya tekak dibuatnya...,-1,bau tengik loya tekak apa teruk peel demam pre...,-1,bau tengik loya tekak apa teruk peel demam pre...,-1
5,Aku rasa aku kena mula jaga pemakanan. Gastrik...,-1,kena jaga pemakanan gastrik ini teruk makan ke...,-1,kena jaga pemakanan gastrik ini teruk makan ke...,-1
6,Apa dia letak sia kat makanan aku. Balik sakit...,-1,letak sia sakit kepala teruk ini,-1,letak tak_sia sakit kepala teruk ini,-1
7,@KuaiMuhamad Tak jgk.. Lama gak x kena food po...,-1,tak juga gak tidak kena food poisoning just my...,-1,tak tak_juga gak tidak tak_kena food poisoning...,-1
8,LoL jual makanan tak sedap pastu mahal lak tu ...,-1,jual tak sedap mahal pula itu haiss,0,jual tak tak_sedap mahal pula itu haiss,-1
9,Makanan dia tak berapa sedap la for me. Dekat ...,-1,tak sedap lah untuk me surau disediakan,1,tak tak_sedap lah untuk me surau disediakan,-1


In [16]:
df.to_csv('done10.csv')

In [17]:
predicted=df.without_valenceshifter.astype(int)
from sklearn.metrics import confusion_matrix
cf =confusion_matrix(df.senti,predicted)
cf

array([[ 88,  59,  36],
       [  7, 120,  10],
       [  8,  66, 206]])

In [18]:
import pandas as pd
exp_series = pd.Series(df.senti)
pred_series = pd.Series(predicted)
pd.crosstab(exp_series, pred_series, rownames=['Actual'], colnames=['Predicted'],margins=True)

Predicted,-1,0,1,All
Actual,,,,
-1,88,59,36,183
0,7,120,10,137
1,8,66,206,280
All,103,245,252,600


In [19]:
predicted1=df.with_valenceshifter.astype(int)
from sklearn.metrics import confusion_matrix
cf =confusion_matrix(df.senti,predicted1)
cf

array([[113,  53,  17],
       [  6, 121,  10],
       [  8,  68, 204]])

In [20]:
import pandas as pd
exp_series = pd.Series(df.senti)
pred_series = pd.Series(predicted1)
pd.crosstab(exp_series, pred_series, rownames=['Actual'], colnames=['Predicted'],margins=True)

Predicted,-1,0,1,All
Actual,,,,
-1,113,53,17,183
0,6,121,10,137
1,8,68,204,280
All,127,242,231,600


In [21]:
df['senti'].value_counts().plot.bar(title='Distribution of sentiment polarity of our dataset')

In [22]:
df1=df[['tweet','tweets']]
df1

,tweet,tweets
0,turn off betul kedai makan bancuh air tak seda...,turn off makan bancuh air tak sedap ini tak pa...
1,Mende tah aku makan tadi. Tak sedap. *sebab bo...,mende entah makan tak sedap claim makan mengarut
2,"@nikasyrvf Mestii, sini kedai makan bukan seda...",mestii makan bukan sedap haha
3,Tapi milo dekat fast food restaurant tak sedap...,milo fast food restaurant tak sedap
4,Semua makanan bau tengik. Loya tekak dibuatnya...,bau tengik loya tekak apa teruk peel demam pre...
5,Aku rasa aku kena mula jaga pemakanan. Gastrik...,kena jaga pemakanan gastrik ini teruk makan ke...
6,Apa dia letak sia kat makanan aku. Balik sakit...,letak sia sakit kepala teruk ini
7,@KuaiMuhamad Tak jgk.. Lama gak x kena food po...,tak juga gak tidak kena food poisoning just my...
8,LoL jual makanan tak sedap pastu mahal lak tu ...,jual tak sedap mahal pula itu haiss
9,Makanan dia tak berapa sedap la for me. Dekat ...,tak sedap lah untuk me surau disediakan


# df2=df[['tweet','negation']]
df2.head()

In [23]:
df3=df[['tweet','negation']]
df3.head()

,tweet,negation
0,turn off betul kedai makan bancuh air tak seda...,turn off makan bancuh air tak tak_sedap ini ta...
1,Mende tah aku makan tadi. Tak sedap. *sebab bo...,mende entah makan tak tak_sedap claim makan me...
2,"@nikasyrvf Mestii, sini kedai makan bukan seda...",mestii makan bukan tak_sedap haha
3,Tapi milo dekat fast food restaurant tak sedap...,milo fast food restaurant tak tak_sedap
4,Semua makanan bau tengik. Loya tekak dibuatnya...,bau tengik loya tekak apa teruk peel demam pre...


In [24]:
df['tweet'].head(8)

0    turn off betul kedai makan bancuh air tak seda...
1    Mende tah aku makan tadi. Tak sedap. *sebab bo...
2    @nikasyrvf Mestii, sini kedai makan bukan seda...
3    Tapi milo dekat fast food restaurant tak sedap...
4    Semua makanan bau tengik. Loya tekak dibuatnya...
5    Aku rasa aku kena mula jaga pemakanan. Gastrik...
6    Apa dia letak sia kat makanan aku. Balik sakit...
7    @KuaiMuhamad Tak jgk.. Lama gak x kena food po...
Name: tweet, dtype: object